##Image Recogniton
    --NEEDS TO BE CLEANED UP !!
        
Convolutional Neuro-Nets are used to predict the classification of an image. Both the MNist and CIFAR data sets have 10 distinct categories. The MNist images are in black and white. The CIFAR images are in color. 

In [1]:
from mnistReader import mnist
import theano
from theano import tensor as T
import numpy as np
from math import sqrt
from time import time
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from scipy.misc import imsave

###Data

In [3]:
xTrain,xTest,yTrain,yTest = mnist(ntrain=1000, 
                                  ntest=1000, 
                                  onehot=True)

Both train and test sets contain 1000 images each. Both sets can include more images for better predictive results at the cost of training time. 

We can see that train sets have 784 features. This is because there are 28 rows and columns per images -- 28 rows times 28 columns results in 784 pixels. 

The test sets have 10 features. Only one of those features has a value of 1 in a certain index to indicate the category of the image, the remaining features have a value of zero. This is One-Hot encoding. 

In [4]:
print xTrain.shape
print yTrain.shape
print xTest.shape
print yTest.shape

(1000, 784)
(1000, 10)
(1000, 784)
(1000, 10)


###Over Fitting
Over training weights is a constant concern in deep learning. Common over training concerns are discussed in the following Bengio paper: http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf A couple of effective methods of regularization are Drop Out http://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf and Max Pooling http://cs231n.github.io/convolutional-networks/  

###MNist 

In [5]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

srng = RandomStreams()

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def rectify(X):
    return T.maximum(X, 0.)
    #return T.maximum(X, 0.01*X)  #leaky rectifier

def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def dropout(X, p=0.0):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates


# input -> [conv -> pool]*3 -> flat -> FC -> frop -> [FC Soft Max]
# alternative notation --> [FC]*2 OR [relu] -> [soft max]

# (128, 32, 15, 15) (128, 64, 7, 7)

def model(X, w, w2, w3, w4, p_drop_conv, p_drop_hidden):
    l1 = rectify(conv2d(X, w, border_mode='full'))
    #l1 = max_pool_2d(l1a, (2, 2))
    l1 = dropout(l1, p_drop_conv)
    
    l2a = rectify(conv2d(l1, w2))
    l2 = max_pool_2d(l2a, (2, 2))
    l2 = dropout(l2, p_drop_conv)

    l3a = rectify(conv2d(l2, w3))
    l3b = max_pool_2d(l3a, (2, 2))
    l3 = T.flatten(l3b, outdim=2) # flatten - change the dims
    l3 = dropout(l3, p_drop_conv)

    l4 = rectify(T.dot(l3, w4))
    l4 = dropout(l4, p_drop_hidden)

    pyx = softmax(T.dot(l4, w_o))
    return l1, l2, l3, l4, pyx




xTrain = xTrain.reshape(-1, 1, 28, 28)
xTest = xTest.reshape(-1, 1, 28,28)

X = T.dtensor4()
Y = T.fmatrix()

w  = init_weights((32, 1, 3, 3)) # computes 32 convolutions with a 3x3 filter
w2 = init_weights((64, 32, 3, 3)) # computers 64 convolutions on 32 activation maps with a 3x3 filter 
w3 = init_weights((128, 64, 3, 3)) # computers 128 convolutions on 64 activation maps with a 3x3 filter 
w4 = init_weights((128 * 4 * 3 * 3, 625))
w_o = init_weights((625, 10))
# 128 * 4 * 3 * 3 = number of elements
# dropped 1st pooling layer, need to mult. by 4 
# pooling layer reduces number of elements in each pooling window from 4 to 1 
# mult. by 4 balances the dims of removing the pooling layer 

noise_l1, noise_l2, noise_l3, noise_l4, noise_py_x = model(X, w, w2, w3, w4, 0.2, 0.5)
l1, l2, l3, l4, py_x = model(X, w, w2, w3, w4, 0., 0.)
y_x = T.argmax(py_x, axis=1)


cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
params = [w, w2, w3, w4, w_o]
updates = RMSprop(cost, params, lr=0.001)

train = theano.function(inputs=[X, Y], 
                        outputs=cost, 
                        updates=updates, 
                        allow_input_downcast=True)

predict = theano.function(inputs=[X], 
                          outputs=y_x, 
                          allow_input_downcast=True)

for i in range(100):
    for start, end in zip(range(0, len(xTrain), 128), range(128, len(xTrain), 128)):
        cost = train(xTrain[start:end], yTrain[start:end])
        a, b, c, d, e = model(floatX(xTrain[start:end]), w, w2, w3, w4, 0., 0.)
        print a.eval().shape, b.eval().shape, c.eval().shape, d.eval().shape
        print np.mean(np.argmax(yTest, axis=1) == predict(xTest))

(128, 32, 30, 30) (128, 64, 14, 14) (128, 4608) (128, 625)
0.096
(128, 32, 30, 30) (128, 64, 14, 14) (128, 4608) (128, 625)
0.106
(128, 32, 30, 30) (128, 64, 14, 14) (128, 4608) (128, 625)
0.109


KeyboardInterrupt: 

###CIFAR

In [6]:
from cifarHandler import cifar

In [7]:
xTrain, yTrain, xTest, yTest = cifar(nData=1, Normalize=True)

In [8]:
print xTrain.shape
print yTrain.shape
print xTest.shape
print yTest.shape

(10000, 3072)
(10000, 10)
(10000, 3072)
(10000, 10)


In [9]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

srng = RandomStreams()

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def rectify(X):
    return T.maximum(X, 0.)
    #return T.maximum(X, 0.01*X)  #leaky rectifier

def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def dropout(X, p=0.0):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates


# input -> [conv -> pool]*3 -> flat -> FC -> frop -> [FC Soft Max]
# alternative notation --> [FC]*2 OR [relu] -> [soft max]

# (128, 32, 15, 15) (128, 64, 7, 7)

def model(X, w, w2, w3, w4, p_drop_conv, p_drop_hidden):
    l1a = rectify(conv2d(X, w, border_mode='full'))
    l1 = max_pool_2d(l1a, (2, 2))
    l1 = dropout(l1, p_drop_conv)
    
    l2a = rectify(conv2d(l1, w2))
    l2 = max_pool_2d(l2a, (2, 2))
    l2 = dropout(l2, p_drop_conv)

    l3a = rectify(conv2d(l2, w3))
    l3b = max_pool_2d(l3a, (2, 2))
    l3 = T.flatten(l3b, outdim=2) # flatten - change the dims
    l3 = dropout(l3, p_drop_conv)

    l4 = rectify(T.dot(l3, w4))
    l4 = dropout(l4, p_drop_hidden)

    pyx = softmax(T.dot(l4, w_o))
    return l1, l2, l3, l4, pyx




xTrain = xTrain.reshape(-1, 3, 32, 32)
xTest = xTest.reshape(-1, 3, 32, 32)

X = T.dtensor4()
Y = T.fmatrix()

w_o = init_weights((625, 10))
w  = init_weights((32, 3, 3, 3)) # computes 32 convolutions with a 3x3 filter
w2 = init_weights((64, 32, 3, 3)) # computers 64 convolutions on 32 activation maps with a 3x3 filter 
w3 = init_weights((128, 64, 3, 3)) # computers 128 convolutions on 64 activation maps with a 3x3 filter

# when dropping pulling in 1st layer, there are 6272 elements in the final activation volume 
# dropped 1st pooling layer, need to mult. by 4 
# pooling layer reduces number of elements in each pooling window from 4 to 1 
# mult. by 4 balances the dims of removing the pooling layer 

#w4 = init_weights((6272 , 625))

# when pooling in 1st layer, there are 1152 elements in the final activation volume
# 128 * 3 * 3 = number of elements in final activation layer 
# 128 convolutions (depth), each convolution is 3x3
w4 = init_weights((128 * 3 * 3, 625))


noise_l1, noise_l2, noise_l3, noise_l4, noise_py_x = model(X, w, w2, w3, w4, 0.2, 0.5)
l1, l2, l3, l4, py_x = model(X, w, w2, w3, w4, 0., 0.)
y_x = T.argmax(py_x, axis=1)


cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
params = [w, w2, w3, w4, w_o]
updates = RMSprop(cost, params, lr=0.001)

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)

for i in range(100):
    for start, end in zip(range(0, len(xTrain), 128), range(128, len(xTrain), 128)):
        cost = train(xTrain[start:end], yTrain[start:end])
        a, b, c, d, e = model(floatX(xTrain[start:end]), w, w2, w3, w4, 0., 0.)
        print a.eval().shape, b.eval().shape, c.eval().shape, d.eval().shape
        print np.mean(np.argmax(yTest, axis=1) == predict(xTest))

(128, 32, 17, 17) (128, 64, 8, 8) (128, 1152) (128, 625)
0.0986
(128, 32, 17, 17) (128, 64, 8, 8) (128, 1152) (128, 625)
0.0991
(128, 32, 17, 17) (128, 64, 8, 8) (128, 1152) (128, 625)


KeyboardInterrupt: 